In [1]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive import BayesianParameterSampling
from azureml.train.hyperdrive import uniform, choice
from azureml.train.hyperdrive import RandomParameterSampling
from azureml.train.hyperdrive import normal, uniform, choice
import os
from azureml.widgets import RunDetails


In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-136467
Azure region: southcentralus
Subscription id: aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee
Resource group: aml-quickstarts-136467


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
cpu_cluster_name = "janetest"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

Found existing cluster, use it.

Running


In [4]:
# Specify parameter sampler
# ps = BayesianParameterSampling( {
#          "--C": uniform(0.1, 1.0), 
#          "--max_iter": choice(100, 500)
#     }
# )


ps = RandomParameterSampling( {
        '--C': uniform(0.2, 1.0),
        '--max_iter': choice(150, 200, 250)
    }
)


### YOUR CODE HERE ###

# Specify a Policy
slack_factor = 0.3
evaluation_interval = 1 
delay_evaluation = 5
policy = BanditPolicy(evaluation_interval= evaluation_interval, slack_factor=slack_factor, delay_evaluation=delay_evaluation, )
primary_metric_name="accuracy"

if "training" not in os.listdir():
    os.mkdir("./training")


# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory= "./",
              compute_target= cpu_cluster,
              entry_script="train.py" )

# est = Estimator(source_directory='./',
#                    compute_target=compute_target,
#                    entry_script='train.py',
#                    conda_packages=['scikit-learn'])

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
     hyperparameter_sampling=ps,
     policy=policy,
     primary_metric_name=primary_metric_name,
     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
     max_total_runs=16,
     max_concurrent_runs=4)


'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [8]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

hdr = exp.submit(config= hyperdrive_config, show_output=True)
RunDetails(hdr).show()


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [9]:
import joblib
# Get your best run and save the model from that run.
best_run = hdr.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']
print('Best run:', best_run)
print('Metrics:', best_run_metrics)
print('Best accuracy: {}'.format(best_run_metrics['accuracy']))

### YOUR CODE HERE ###

Best run: Run(Experiment: udacity-project,
Id: HD_f0d8a64d-7c34-4800-a4b4-7aa30703517e_11,
Type: azureml.scriptrun,
Status: Finalizing)
Metrics: {'Regularization Strength:': 0.4443480490228587, 'Max iterations:': 200, 'accuracy': 0.9171471927162367}
Best accuracy: 0.9171471927162367


In [10]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

from azureml.data.dataset_factory import TabularDatasetFactory
path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path)

### YOUR CODE HERE ###

In [11]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)


In [12]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric= 'accuracy',
    training_data= ds,
    label_column_name= 'y',
    n_cross_validations= 5)

In [13]:
# Submit your automl run
automl_run = exp.submit(automl_config, show_output=True)
automl_run.wait_for_completion()

### YOUR CODE HERE ###

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_7a76faed-ebd2-4031-9d5e-52d50ab7e2a0

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/Auto

{'runId': 'AutoML_7a76faed-ebd2-4031-9d5e-52d50ab7e2a0',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-01-29T04:15:01.820177Z',
 'endTimeUtc': '2021-01-29T04:47:32.728752Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee","resource_group":"aml-quickstarts-136467","workspace_name":"quick-starts-ws-136467","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":2,"f

In [14]:
RunDetails(automl_run).show()

print('Best run:', best_run)
print('Metrics:', best_run_metrics)
print('Best accuracy: {}'.format(best_run_metrics['accuracy']))


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Best run: Run(Experiment: udacity-project,
Id: HD_f0d8a64d-7c34-4800-a4b4-7aa30703517e_11,
Type: azureml.scriptrun,
Status: Finalizing)
Metrics: {'Regularization Strength:': 0.4443480490228587, 'Max iterations:': 200, 'accuracy': 0.9171471927162367}
Best accuracy: 0.9171471927162367


In [18]:
# Retrieve and save your best automl model.

best_run, fitted_model = automl_run.get_output()

if "outputs" not in os.listdir():
    os.mkdir("./outputs")

output_dir='./outputs/model'
os.makedirs(output_dir, exist_ok=True)
filename = "bankmarketing_model.pkl"
output_path = os.path.join(output_dir, filename)

joblib.dump(fitted_model,filename=output_path )
print("model saved in {}".format(output_path))

### YOUR CODE HERE ###

model saved in ./outputs/model/bankmarketing_model.pkl


In [19]:
automl_config2 = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric= 'accuracy',
    training_data= ds,
    label_column_name= 'y',
    n_cross_validations= 5,
enable_voting_ensemble=False,
        enable_stack_ensemble=False)

In [20]:
automl_run = exp.submit(automl_config2, show_output=True)
automl_run.wait_for_completion()
RunDetails(automl_run).show()


No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_5321cdf7-4365-4d8f-92f2-b2affda826c1

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/Auto

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [ ]:
cpu_cluster.delete()